In [2]:
import os
import h5py
import matplotlib.pyplot as plt
import numpy as np
from numba import jit 

In [3]:
def read_h5_jit(filename):
    # Open the HDF5 file
    with h5py.File(filename, 'r') as opf:
        x = opf['X'][()]
        y = opf['Y'][()]
        toa = opf['ToA'][()]
        tot = opf['ToT'][()]
        shot = opf['Frame'][()]
        cent = np.zeros((x.shape))
    # Turn data into single array
    all_data = np.vstack((tot, toa, x, y,shot,cent)).T
    
    # Get individual shot data from array
    shot_data = []
    start_idx = 0
    array_max = 0
    for shot_index in np.unique(shot):
        # Find the index to the right of the last value
        slice_end_idx = np.searchsorted(shot, shot_index, side='right') 
        shot_array = all_data[start_idx:slice_end_idx]
        if shot_array.shape[0] > array_max:
            array_max = shot_array.shape[0]
        shot_data.append(shot_array)
        start_idx = slice_end_idx

    # Pad the arrays so they all have the same length for jit
    for index, shot in enumerate(shot_data):
        pad_size = array_max - shot.shape[0]
        pad_array = np.zeros((pad_size,6))
        shot_data[index] = np.vstack((shot, pad_array))

    return np.array(shot_data)

In [4]:
#ToT/ToA/x/y/shot_number/0
@jit(nopython = True)
def centroid_shots(input_array, time_win, min_t, max_t, min_cluster_size, max_cluster_size):
    event_array = []
    event_no = 0
    event_counter = 0

    for shot_index, shot_array in enumerate(input_array):
        image = np.zeros((256,256)) # used to keep track of where the hits for on each shot
        
        # Search events in shot to find an uncounted hit
        for i, event in enumerate(shot_array):
            if np.sum(event) == 0: continue # Skip array padding
            # search final column of input array, within which 0 -> uncounted, 1 -> counted
            if event[5] == 1: continue
                
            # Check event is in time range specified
            i_t = event[1]
            ToT = event[0]
            if min_t <= i_t <= max_t:
                shot_array[i][5] = 1 # mark hit as found
                cluster_array = np.zeros((20300, 3))
                size  = 1 # reset cluster size
                event_no += 1
                i_row = int(event[2]) # x coordinate
                i_col = int(event[3]) # y coordinate
                cluster_array[size - 1, 0] = i_row # Add hit to cluster array
                cluster_array[size - 1, 1] = i_col 
                cluster_array[size - 1, 2] = i_t
                min_t_event = i_t
                max_t_event = i_t + time_win
                image[i_row, i_col] = event_no # mark where the hit was in the image
                    
                # Search for connected hits
                found_neighbour = True
                for _ in range(max_cluster_size + 10): # search for a maximum of (max_cluster_size + 10) additional hits
                    if found_neighbour:
                        found_neighbour = False
                        
                        for j in range(i, len(shot_array)):
                            found_j = False
                            if shot_array[j, 5] == 0:
                                
                                # Check hit j is within the time window of the initial hit i
                                j_t = shot_array[j, 1]
                                if min_t_event <= j_t <= max_t_event:
                                    
                                    j_row = int(shot_array[j, 2])
                                    j_col = int(shot_array[j, 3])
                                    
                                    # Check the found hit j is close to the cluster in the image
                                    for j_row2 in range(j_row - 1, j_row + 2):
                                        if found_j == False:
                                            for j_col2 in range(j_col - 1, j_col + 2 ):
                                                if (j_row2 > 255) or (j_col2 > 255): continue
                                                if image[j_row2, j_col2] == event_no:
                                                    
                                                    shot_array[j, 5] = 1 # mark event as found
                                                    size += 1 # increment cluster size
                                                    image[j_row, j_col] = event_no # mark new hit in the image
                                                    cluster_array[size - 1, 0] = j_row # Add hit to cluster array
                                                    cluster_array[size - 1, 1] = j_col
                                                    cluster_array[size - 1, 2] = j_t

                                                    # Reset these variables if a neighbouring hit was found to keep on searching for more
                                                    found_neighbour = True
                                                    found_j = True
                                                    break
                    
                # Convert cluster into an event by applying equation to find cluster centre
                if min_cluster_size <= size <= max_cluster_size: # check cluster size is within bounds
                    
                    x_sum = 0
                    y_sum = 0
                    t_sum = 0
                    
                    # Cluster_array isn't reintialised, new data overwrites old, so only read out data in the array 
                    # saved for the current cluster
                    for k in range(size): 
                        cx = cluster_array[k, 0]
                        cy = cluster_array[k, 1]
                        ct = cluster_array[k, 2]
                        time = ct - i_t + 1
                        x_sum += cx/time
                        y_sum += cy/time     
                        t_sum += 1/time
                    
                    t_spread = max(cluster_array[:size, 2]) - min(cluster_array[:size, 2])

                    event_array.append([shot_index, int(x_sum/t_sum + 0.5), int(y_sum/t_sum + 0.5), i_t, ToT, size, t_spread])
                    # event_array.append([shot_index, int(x_sum/t_sum + 0.5), int(y_sum/t_sum + 0.5), i_t, ToT, size, int(t_sum)])
                    event_counter += 1

    return event_array


In [5]:
directory = r'Q:\Cameras\TimePix\PyPix-Advacam-TPX3'
filename = '290824_02_3kvRepeller_0threshold_100Hz_300s_0000.h5'
time_win = 10*25 # maximum time ns in a cluster for centroiding
max_ion_count = 500000000
min_t, max_t = 0*25, 100000# timewindow to centroid over in nanoseconds
min_cluster_size, max_cluster_size = 1, 40

fp = os.path.join(directory,filename)
print('Reading data from file.')
input_array = read_h5_jit(fp)
print('Centroiding data.')
output_array = centroid_shots(input_array, time_win, min_t, max_t, min_cluster_size, max_cluster_size)
output_array = np.array(output_array)

# Plot counts vs. cluster size 
sizes, counts = np.unique(output_array[:,5], return_counts = True)
fig, ax = plt.subplots(figsize=(9,5))
ax.bar(sizes, counts)
ax.set_xlabel('Cluster size / pixels', fontsize = 16)
ax.set_ylabel('Counts', fontsize = 16)
plt.show()

# Plot counts vs. time range
sizes, counts = np.unique(output_array[:,6], return_counts = True)
fig, ax = plt.subplots(figsize=(9,5))
ax.bar(sizes, counts)
ax.set_xlabel('Time range / bins', fontsize = 16)
ax.set_ylabel('Counts', fontsize = 16)
plt.show()

np.savetxt(f'{fp[:-5]}_centroided.csv', output_array[:,0:5], delimiter=",", fmt=['%i','%i','%i','%.4f','%i'], header='Shot,X,Y,ToA,ToT')
# np.savetxt(f'{fp[:-5]}_centroided2.csv', output_array[:,0:5], delimiter=",", fmt=['%i','%i','%i','%.4f','%i'], header='Shot,X,Y,ToA,ToT')
    
print('Finished')

Reading data from file.


MemoryError: Unable to allocate 1.94 GiB for an array with shape (29369, 1481, 6) and data type float64